# Scraping FDA Warnings

Web scraping is the process of grabbing information off the internet. While it's powerful, it's also complex; just as all websites look and act differently, there's no one-size-fits-all solution to making it happen. 

Today we'll be going over some basic concepts while using the [FDA's Warning Letters](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm). We'll start by grabbing the data we can see on the page. If time allows, we'll grab data for the whole year. And if this is the fastest-moving one-hour scraping course in the history of IRE, we'll look at how to grab ALL years. 

To start out with, we're going to grab three libraries that will help us through this process:

In [ ]:
import requests
from bs4 import *
import csv

### WTF was that
Python is a programming language, no better or worse than Ruby, PHP, Perl or anything else. We're using it because it's pretty intuitive, and, well, it's what I'm best at. One of the great parts about Python is that there are a lot of handy tools already made for it by people much smarter than us. All we have to do is bring them in, which we do with the handy `import` command.

In this case, we're bringing in three libraries:
* [`requests`](http://docs.python-requests.org/en/master/) is a handy third-party library for making HTTP requests. It does the same thing your browser does when you type in a URL and hit enter -- sends a message to a server and requests a copy of the page. We'll use it to get Python the right pages to scrape.
* [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a library you can use to parse HTML into data types that Python can work with. We'll use it to grab the contents of the page we want.
* [`csv`](https://docs.python.org/2/library/csv.html) is a built-in library that reads and writes csv, which is of course God's chosen data format. Here, we'll be using it to spit out a csv for analysis.

In [ ]:
url = "https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, "html5lib")
table = soup.find('table')

### Requests and BeautifulSoup, brothers in arms
Let's unpack what we just did there. First, we defined our url as the start page for this scrape. Now, rather than type the whole url, we can just say "url" and be good to go. 
We've set up a variable called "response" to hold our webpage in Python, and something called "html" which is the actual code that makes the page. 
We've created something called "soup," which is a BeautifulSoup representation of the page. This will let us pull it apart using the powerful parsing library.
This process of getting a page and turning it into a BeautifulSoup object is really common in scraping, and for good reason. In the last bit, we use BeautifulSoup to track down the lone table on the page -- the very part we want to turn into a csv. Let's check that out:

In [ ]:
print(table)

Wouldja Look at that! This is the power of BeautifulSoup: it understands html and translates to Python. By looking for the "table" tag in our soup, we identified the whole table of information we want. Now that we have it identified, we can start working on separating the information into the right shape to make a csv.

### Thinking about data format
Our end result is a csv. Because we have the handy-dandy `csv` module, we know that one easy way to get it done is to create a list, one of the Python data types (more on that in [IRE's fabulous intro to Python repo](https://github.com/cjwinchester/ire-2018-python-for-data-analysis/blob/master/reference/Python%20data%20types%20and%20basic%20syntax.ipynb)).
To get there, we're going to create an empty list. Then we'll fill it in with the good stuff.


In [ ]:
list_of_rows = []
for row in table.findAll('tr')[1:]:
    list_of_cells = []
    for cell in row.findAll('td'):
        text = cell.text.strip()
        list_of_cells.append(text)
    list_of_rows.append(list_of_cells)
print(list_of_rows)

### Grabbing the data
Let's step back through it piece by piece:
* `list_of_rows` starts out as an empty list. This is basically a holding place where we'll put our stuff. 
* That `for` section and the associated indentation, indicates a loop. Anytime you see that, we're doing something a set number of times.
* The `for row in table.findAll('tr')` section is going into our table and defining what, exactly, we're iterating over. In this case, it's each "tr" -- the html markup for a table row.
* Every time we look at a row, we're going to store it in a temporary container called `list_of_cells`
* Then, for every cell ('td' -- a table division in html) we're going to grab the text and show it into our `list_of_cells`
* Once we've grabbed everything from the row, we'll put the whole thing into our `list_of_rows` and move on to the next one.
    
Now that we have a list of lists, let's write it:


In [ ]:
outfile = open('fdaresults.txt', 'w')
writer = csv.writer(outfile)
writer.writerows(list_of_rows)
outfile.close()

### Writing the data
This part, too, is fairly boilerplate anytime you want to turn a list of lists into a csv:
* Open a file. In this case, it's called "fdaresults.txt"
* The `w` argument indicates that we're opening the file to write to.
* Set up a way to write to that file using the `csv` library
* Write the dang thing
  
There we go! We successfully turned a table form a webpage into a csv. Underwhelmed? Me too. [In our next section](https://github.com/mattwynn1/IRE-scraping-2018/blob/master/Basic%20scraping%20part%202%2C%20basic%20interaction.ipynb), we're going to take what we've learned and extrapolate a bit to get ALL the records from 2018.